# Сбор данных с сайта РИА Новости

*Ноутбук сделан по мотивам ноутбука авторства Марии Гольдштейн и Кирилла Долматова.*

Установим и импортируем необходимые библиотеки.

In [ ]:
%%capture
!pip install selenium

In [ ]:
import re
import time
import datetime
import pandas as pd
import warnings
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from dataclasses import dataclass

warnings.filterwarnings("ignore", category=FutureWarning)

Далее зададим необходимые константы.

`DEPTH` - число страниц для скрапинга

In [ ]:
SLEEP = 1
DEPTH = 85
BASE_URL = "https://ria.ru/"
TOPICS = ["economy", "society", "incidents", "science", "culture"]

In [ ]:
@dataclass
class Article:
    id: str = None
    url: str = None
    title: str = None
    subtitle: str = None
    content: str = None
    datetime: str = None

In [ ]:
# set webdriver params
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--blink-settings=imagesEnabled=false")
chrome_options.add_argument("headless")
chrome_options.add_argument("no-sandbox")
chrome_options.add_argument("disable-dev-shm-usage")
driver = webdriver.Chrome(options=chrome_options)

Посмотрим на функцию для скачивания страниц.

In [ ]:
def get_pages():

    """Load and scroll pages"""

    items, topics_order = [], []

    for topic in tqdm(TOPICS):
        try:
            old_size = len(items)
            URL = BASE_URL + topic
            driver.get(URL)
            time.sleep(SLEEP)

            # push to list 20 next articles
            driver.execute_script(
                "document.getElementsByClassName('list-more')[0].click()"
            )
            time.sleep(1)

            # scroll page to automatically load more articles
            for i in tqdm(range(DEPTH), leave=False):
                try:
                    driver.execute_script(
                        f"window.scrollTo(0, document.body.scrollHeight - 1200)"
                    )
                    time.sleep(1)
                except:
                    pass

            # find all pages
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            scope = soup.find(
                "div", {"class": "list", "itemtype": "http://schema.org/ItemList"}
            )
            items += scope.find_all("div", {"class": "list-item"})

            # number of pages can not be multiple of deepth*20
            # that's why we count topics_order dynamically
            new_size = len(items)
            if new_size > old_size:
                topics_order.extend([topic] * (new_size - old_size))
        except:
            pass

    return items, topics_order

Посмотрим на функцию для парсинга скачанных страниц.

In [ ]:
def parse_page(page):
    """Extract from page desired fields"""

    # Create article data class object
    article = Article()

    # article url
    article.url = page.find("a", {"class": "list-item__image"})["href"]

    # article id
    s = re.findall(r"\d+.html", article.url)[0]
    article.id = s[: s.find(".")]

    # load page
    driver.get(article.url)
    time.sleep(SLEEP)
    html = driver.page_source

    # article source
    source = article.url[8 : article.url.find(".")]

    # article object
    soup = BeautifulSoup(html, "html.parser")
    obj = soup.find(
        "div",
        {
            "class": lambda x: x and (x.find(f"article m-article m-{source}") > -1),
            "data-article-id": article.id,
        },
    )

    if not obj:
        obj = soup.find(
            "div",
            {
                "class": lambda x: x and (x.find(f"article m-video m-{source}") > -1),
                "data-article-id": article.id,
            },
        )

    # process article title
    title = obj.find("div", {"class": "article__title"})
    title_2 = obj.find("h1", {"class": "article__title"})

    if title:
        article.title = title.text
    else:
        article.title = title_2.text if title_2 else ""

    # article subtitle
    subtitle = obj.find("h1", {"class": "article__second-title"})
    article.subtitle = subtitle.text if subtitle else ""

    # article content
    article.content = obj.find(
        "div", {"class": "article__body js-mediator-article mia-analytics"}
    ).text

    # article datetime
    article.datetime = obj.find("div", {"class": "article__info-date"}).find("a").text

    # article number of views
    # article.views = int(obj.find('span', {'class': 'statistic__item m-views'}).text)

    return article

Скачаем новости (количество регулируется константой DEPTH выше).

In [ ]:
# get pages and topics
pages, topics_order = get_pages()

100%|██████████| 5/5 [09:24<00:00, 112.90s/it]


Сохраним результат в файл.

In [ ]:
with open("ria_pages.txt", "w", encoding="utf-8") as f:
    for page in pages:
        f.write("%s\n" % page)

## Вопрос 1

Сколько страниц `pages` мы спарсили?

In [ ]:
len(pages)

NameError: ignored

Распарсим каждую страницу и достанем из нее необходимые атрибуты:

* в цикле примените функцию `parse_page` к каждой странице и сохраните результат в объект `res`

* затем добавьте `res` в список `data`


Для скорости распарсим только первые 100 страниц.

In [ ]:
data, topics_order_fixed = [], []

for num, page in enumerate(tqdm(pages)):
    try:
        res = parse_page(page)
        data.append(res)

        topics_order_fixed.append(topics_order[num])
    except:
        pass

driver.close()

 54%|█████▎    | 3874/7208 [6:39:08<6:15:23,  6.76s/it]

Преобразуем в датафрейм полученные данные.

## Вопрос 2

Сколько столбцов в таблице `df`?

In [ ]:
df = pd.DataFrame(data=data)

df.shape

Добавьте в таблицу `df` столбец `topic`, содержащий список `topics_order_fixed`

In [ ]:
len(topics_order_fixed)

In [ ]:
df["topic"] = topics_order_fixed

## Вопрос 3

Какой `topic` имеет новость из первой строки таблицы?

In [ ]:
df.head()

Сохраним скачанные данные в файл.

In [ ]:
df.to_pickle("df_ria.p", compression="gzip")

In [ ]:
df.to_csv("df_ria_80.csv")

## Вопрос 4

Есть ли пропуски в каких-либо столбцах в таблице `df`?

In [ ]:
df.isna().any().any()